> 41271118H 柯亮宇

<!--網頁爬蟲-->
學習分析期中作業二

# 第一部分：Selenium

In [1]:
#@title 🚀 下載套件

# 先下載所有要使用到的套件並導入
from IPython.display import clear_output, HTML
!curl -o- https://gist.githubusercontent.com/samko5sam/27dc5983bbdd37413eaf5a3d32f52108/raw/0bca202eb07864d8e6f10de8b18726abea8b705d/selenium.sh | bash

clear_output()
from bs4 import BeautifulSoup
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random

In [2]:
#@title 🔗 取得選書連結清單
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100) # 設定等待時間

outputContent = ''

urls = []
driver.get("https://ebook.moc.gov.tw/category/readers/")
articlesList = driver.find_elements(By.CSS_SELECTOR, ".elementor-posts .elementor-post") # 找到 container
print(len(articlesList))
for i in range(len(articlesList)):
  articleLink = articlesList[i].find_element(By.CSS_SELECTOR, ".elementor-posts-container > .elementor-post > .elementor-post__card > .elementor-post__thumbnail__link").get_attribute("href") # 取得連結
  outputContent += '{}: <a href="{}">{}</a><br>'.format(i,articleLink,articleLink)
  urls.append(articleLink)

driver.quit()
HTML(outputContent)

12


In [3]:
#@title 🌐 取得選書內容
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100)

urls = urls[1:5] # 爬取四位選書人的書

# htmlContent用來輸出爬取內容
htmlContent = '<h3>選書</h3><table><tr><th>選書人</th><th>書名</th><th>作者</th><th>出版社</th><th>譯者/繪者</th></tr>'

with open("selectedBooks.csv", "w") as csvfile:
    csvfile.write('')
    csvfile.close()

def getAndAppendWebContent(url):
  myHtmlContent = ''
  time.sleep(random.randint(0,3))
  driver.get(url) # 打開網頁

  soup = BeautifulSoup(driver.page_source, "lxml")
  fp = open("index.html", "w", encoding="utf8")
  fp.write(soup.prettify())
  fp.close()
  booksList = driver.find_elements(By.CLASS_NAME, "elementor-section")
  for i in range(len(booksList)):
    if i >= 10 and i <= 14: # 因為會選到其他內容，所以用index來判斷
      bookTitle = booksList[i].find_element(By.CSS_SELECTOR, ".elementor-column > .elementor-widget-wrap > .elementor-element > .elementor-widget-container > h5").text
      bookInfoP = booksList[i].find_elements(By.CSS_SELECTOR, ".elementor-column > .elementor-widget-wrap > .elementor-element > .elementor-widget-container > p")
      bookAuthor = bookInfoP[0].text[3:].split("　")[0] # 有一些作者欄還有繪者或譯者，所以用空格來分離
      additionalAuthorsList = bookInfoP[0].text[3:].split("　")
      additionalAuthorsList.pop(0)
      bookAdditionalAuthor = " ".join(additionalAuthorsList)
      bookPublisher = bookInfoP[1].text[4:] # 移除"出版社："
      myHtmlContent += '<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>'.format(driver.title[:-15],bookTitle,bookAuthor,bookPublisher,bookAdditionalAuthor)
      if bookAdditionalAuthor == "":
        bookAdditionalAuthor = "null"
      with open("selectedBooks.csv", "a") as csvfile:
        csvfile.write('{},{},{},{},{}'.format(driver.title[:-15],bookTitle,bookAuthor,bookPublisher,bookAdditionalAuthor))
        csvfile.close()
    if i >= 10 and i <= 13:
      with open("selectedBooks.csv", "a") as csvfile:
        csvfile.write("\n")
        csvfile.close()
  return myHtmlContent

for i in range(len(urls)):
  htmlContent += getAndAppendWebContent(urls[i])
  if i != len(urls) - 1:
    with open("selectedBooks.csv", "a") as csvfile:
      csvfile.write("\n")
      csvfile.close()

driver.quit()
htmlContent += '</table>'
HTML(htmlContent)

選書人,書名,作者,出版社,譯者/繪者
曾寶儀,神鵰俠侶（新修文庫版）,金庸,遠流出版事業股份有限公司,
曾寶儀,呼吸，為了療癒：全新的呼吸科學與醫學，透過清醒的呼吸，徹底轉化身心,楊定一,天下生活,
曾寶儀,跟一棵樹聊天，聽他的人生哲學：春花媽的植物療癒旅程,春花媽,日出出版,繪/Soupy Tang湯舒皮
曾寶儀,死過一次才學會愛：艾妮塔的瀕死重生奇蹟,艾妮塔．穆札尼（Anita Moorjani）,橡實文化出版,譯/隋芃
曾寶儀,人生最大的成就，是成為你自己,曾寶儀,天下生活,
陳艾琳,只要我長大,李光福,小魯文化事業股份有限公司,
陳艾琳,謝謝妳是我媽媽,宋貞林,境好出版,譯/陳品芳
陳艾琳,為何三歲開始說謊？：探究心智起源，解開0-15歲孩子的大腦與行為之謎,謝伯讓,親子天下股份有限公司,
陳艾琳,你的孩子不奇怪：改變，從理解孩子的奇怪開始,李介文,時報文化出版企業股份有限公司,
陳艾琳,AI如何重塑教育：ChatGPT來了！讓孩子活出熱情，啟動真探究的內在學習,陳雅慧；賓靜蓀；溫怡玲；親子天下,親子天下股份有限公司,


# 第二部分：Request

In [11]:
#@title 🚀 下載套件
import sys
import subprocess
from IPython.display import clear_output, HTML
from bs4 import BeautifulSoup

# 使用check_call函數執行指定的命令，該函數會等待命令完成並檢查返回碼。
# 在這裡，指定的命令是一個列表，包含要在命令行中執行的命令以及相應的參數。其中：
# -`sys.executable`提供Python解釋器的路徑，以確保在正確的環境中執行pip。
# -`-m pip` 表示使用Python的pip模組。
# -`install cssselect` 是指定要安裝的模組。
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'cssselect'])
clear_output()

In [8]:
#@title 🔗 取得選書連結清單


# 和上面 Selenium 一樣，只是使用 Requests 達成


import requests
from lxml import html
urls = []
r = requests.get("https://ebook.moc.gov.tw/category/readers/")
tree = html.fromstring(r.text)
articlesList = tree.cssselect(".elementor-posts-container > .elementor-post > .elementor-post__card > .elementor-post__thumbnail__link")
# print(len(articlesList))
for i in range(len(articlesList)):
  articleLink = articlesList[i].get("href")
  print(i, articleLink)
  urls.append(articleLink)

0 https://ebook.moc.gov.tw/2024/05/01/ngselections/
1 https://ebook.moc.gov.tw/2024/02/20/bowieselections/
2 https://ebook.moc.gov.tw/2024/01/29/ailingselections/
3 https://ebook.moc.gov.tw/2024/01/15/lienselections/
4 https://ebook.moc.gov.tw/2024/01/15/anitaselections/
5 https://ebook.moc.gov.tw/2023/12/15/hsiehtunglinselections/
6 https://ebook.moc.gov.tw/2023/12/15/plainlawselections/
7 https://ebook.moc.gov.tw/2023/12/15/incredivilleselections/
8 https://ebook.moc.gov.tw/2023/11/16/fanchifeiselections/
9 https://ebook.moc.gov.tw/2023/11/15/cherselections/
10 https://ebook.moc.gov.tw/2023/10/31/ryanhsiehselections/
11 https://ebook.moc.gov.tw/2023/10/31/chuckchuselections/


In [9]:
#@title 🌐 取得選書內容


# 和上面 Selenium 一樣，只是使用 Requests 達成


urls = urls[1:5] # 只取四個選書人

htmlContent = '<h3>選書</h3><table><tr><th>選書人</th><th>書名</th><th>作者</th><th>出版社</th><th>譯者/繪者</th></tr>'

with open("selectedBooks.csv", "w") as csvfile:
    csvfile.write('')
    csvfile.close()

def getAndAppendWebContent(url):
  myHtmlContent = ''
  r = requests.get(url)
  soup = BeautifulSoup(r.text, "lxml")
  webTitle = soup.select("title")[0].text
  booksInfo = soup.select(".elementor-container .elementor-section")
  for i in range(len(booksInfo)):
    if i >= 5 and i <= 9:
      bookTitle = booksInfo[i].select(".elementor-widget-container > h5")
      bookInfoP = booksInfo[i].select(".elementor-column > .elementor-widget-wrap > .elementor-element > .elementor-widget-container > p")
      if (len(bookTitle)):
        bookTitle = bookTitle[0].text
        bookAuthor = bookInfoP[0].text[3:].split("　")[0]
        additionalAuthorsList = bookInfoP[0].text[3:].split("　")
        additionalAuthorsList.pop(0)
        bookAdditionalAuthor = " ".join(additionalAuthorsList)
        bookPublisher = bookInfoP[1].text[4:]
        myHtmlContent += '<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>'.format(webTitle[:-15],bookTitle,bookAuthor,bookPublisher,bookAdditionalAuthor)
        if bookAdditionalAuthor == "":
          bookAdditionalAuthor = "null"
        with open("selectedBooks.csv", "a") as csvfile:
          csvfile.write('{},{},{},{},{}'.format(webTitle[:-15],bookTitle,bookAuthor,bookPublisher,bookAdditionalAuthor))
          csvfile.close()
    if i >= 5 and i <= 8:
      with open("selectedBooks.csv", "a") as csvfile:
        csvfile.write("\n")
        csvfile.close()
  return myHtmlContent

for i in range(len(urls)):
  htmlContent += getAndAppendWebContent(urls[i])
  if i != len(urls) - 1:
    with open("selectedBooks.csv", "a") as csvfile:
      csvfile.write("\n")
      csvfile.close()

htmlContent += '</table>'
HTML(htmlContent)

選書人,書名,作者,出版社,譯者/繪者
曾寶儀,神鵰俠侶（新修文庫版）,金庸,遠流出版事業股份有限公司,
曾寶儀,呼吸，為了療癒：全新的呼吸科學與醫學，透過清醒的呼吸，徹底轉化身心,楊定一,天下生活,
曾寶儀,跟一棵樹聊天，聽他的人生哲學：春花媽的植物療癒旅程,春花媽,日出出版,繪/Soupy Tang湯舒皮
曾寶儀,死過一次才學會愛：艾妮塔的瀕死重生奇蹟,艾妮塔．穆札尼（Anita Moorjani）,橡實文化出版,譯/隋芃
曾寶儀,人生最大的成就，是成為你自己,曾寶儀,天下生活,
陳艾琳,只要我長大,李光福,小魯文化事業股份有限公司,
陳艾琳,謝謝妳是我媽媽,宋貞林,境好出版,譯/陳品芳
陳艾琳,為何三歲開始說謊？：探究心智起源，解開0-15歲孩子的大腦與行為之謎,謝伯讓,親子天下股份有限公司,
陳艾琳,你的孩子不奇怪：改變，從理解孩子的奇怪開始,李介文,時報文化出版企業股份有限公司,
陳艾琳,AI如何重塑教育：ChatGPT來了！讓孩子活出熱情，啟動真探究的內在學習,陳雅慧；賓靜蓀；溫怡玲；親子天下,親子天下股份有限公司,


In [27]:
#@title 🔗 取得電子書排行榜
import requests
from lxml import html
import re
with open("chart.csv", "w") as csvfile:
    csvfile.write('')
    csvfile.close()
htmlContent = '<h3>熱銷排行</h3><table><tr><th>排行</th><th>書名</th><th>作者</th><th>出版社</th></tr>'
r = requests.get("https://www.pubu.com.tw/hot/ebook")
tree = html.fromstring(r.text)
bookTitles = tree.cssselect("#search-list-content > div > article > div > div.info-block > div.info-project > div > h3 > a")
bookAuthors = tree.cssselect("#search-list-content > div > article > div > div.info-block > div.info-project > div > div:nth-child(3) > a")
bookPublishers = tree.cssselect("#search-list-content > div > article > div > div.info-block > div.info-project > div > div:nth-child(2) > a")
for i in range(len(bookTitles)):
  htmlContent += '<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>'.format(i+1, bookTitles[i].attrib["title"], re.sub(r'[\n\t\s]*', '', bookAuthors[i].text), re.sub(r'[\n\t\s]*', '', bookPublishers[i].text))
  with open("chart.csv", "a") as csvfile:
      csvfile.write('{},{},{},{}'.format(i+1, bookTitles[i].attrib["title"], re.sub(r'[\n\t\s]*', '', bookAuthors[i].text), re.sub(r'[\n\t\s]*', '', bookPublishers[i].text)))
      if i!=len(bookTitles)-1:
        csvfile.write("\n")
      csvfile.close()

htmlContent += '</table>'
HTML(htmlContent)

排行,書名,作者,出版社
1,【軍公教、學生專屬優惠】Pubook SE 7.8吋閱讀器,湛天出版,湛天出版
2,巨人思維,巨人傑,天下文化
3,聖潔的愛 Your Holy Love 樂譜,火把音樂TorchMusic,火把音樂
4,文創風1211國師的愛徒 下,莫顏,狗屋出版社
5,文創風1210國師的愛徒 上,莫顏,狗屋出版社
6,01 奪回這世代 Reclaim This Generation 樂譜,火把音樂TorchMusic,火把音樂
7,蛤蟆先生去看心理師（暢銷300萬冊！英國心理諮商經典）,羅伯·狄保德(RobertdeBoard),三采文化股份有限公司
8,詭軼紀事‧捌：噬人詭衣櫃,Div（另一種聲音）,笭菁工作室
9,一切都會好轉的：查理的百歲人生教會我的事,笭菁,時報出版
10,我，刀槍不入：從街頭魯蛇到海豹突擊隊，掌控心智、力抗萬難的奇蹟,龍雲,方智
